In [ ]:
import os
import platform
import logging

os.environ['KMP_DUPLICATE_LIB_OK'] = 'TRUE'

import tqdm
import cv2
import numpy as np
import pandas as pd

from street_signs_identify.utils.x_image_loader import XImageLoader
import street_signs_identify.utils.check_overlap as co

from street_signs_identify.detector.fr_detector import FRDetector
from street_signs_identify.detector.craft_detector import CRAFTDetector

In [ ]:
LOGGER = logging.getLogger(__name__)

In [ ]:
if platform.system() == "Linux":
    model_path = "./street_signs_identify/detector/saved_models/frcnn.pth"
    folder_name = "./dataset/"
    data_path = os.path.join(folder_name, "2Penghu")
    output_folder = os.path.join(folder_name, "frcnn_craft_cropped_2Penghu")
    output_folder_gt = os.path.join(output_folder, "ref_gt")
    output_folder_filtered_pred = os.path.join(output_folder, "filtered_pred")
    output_folder_craft_pred = os.path.join(output_folder, "craft_pred")

else:
#     folder_name = "train"
#     model_path = r"C:\Users\MAGIC\Desktop\model\frcnn\best_map_frcnn_FrzModel.pth"
#     data_path = f'C:\\Users\\MAGIC\\Ching\\survey\\plan\\rawdata_all\\7types_v2\\x\\{folder_name}'
#     output_folder = f'C:\\Users\\MAGIC\\Ching\\survey\\plan\\rawdata_all\\7types_v2\\xfc\\{folder_name}'
#     output_folder_gt = f'C:\\Users\\MAGIC\\Ching\\survey\\plan\\rawdata_all\\7types_v2\\xfc_gt\\{folder_name}'
#     output_folder_filtered_pred = f'C:\\Users\\MAGIC\\Ching\\survey\\plan\\rawdata_all\\7types_v2\\xfc_filter\\{folder_name}'
#     output_folder_craft_pred = f'C:\\Users\\MAGIC\\Ching\\survey\\plan\\rawdata_all\\7types_v2\\xfc_fast_craft\\{folder_name}'
    
    # demo
    model_path = r"C:\Users\MAGIC\Desktop\model\frcnn\best_map_frcnn_FrzModel.pth"
    data_path = r'C:\Users\MAGIC\Ching\survey\plan\rawdata_all\7types_v2\demo'
    output_folder = f'C:\\Users\\MAGIC\\Ching\\survey\\plan\\rawdata_all\\7types_v2\\demo\\xfc'
    output_folder_gt = f'C:\\Users\\MAGIC\\Ching\\survey\\plan\\rawdata_all\\7types_v2\\demo\\xfc_gt'
    output_folder_filtered_pred = f'C:\\Users\\MAGIC\\Ching\\survey\\plan\\rawdata_all\\7types_v2\\demo\\xfc_filter'
    output_folder_craft_pred = f'C:\\Users\\MAGIC\\Ching\\survey\\plan\\rawdata_all\\7types_v2\\demo\\xfc_fast_craft'



os.makedirs(output_folder, exist_ok=True)
os.makedirs(output_folder_gt, exist_ok=True)
os.makedirs(output_folder_filtered_pred, exist_ok=True) 
os.makedirs(output_folder_craft_pred, exist_ok=True)

In [ ]:
frcnn_detector = FRDetector(model_ckpt=model_path, parse_label=True)
craft_detector = CRAFTDetector()
x_image_loader = XImageLoader(data_path, annotation_file="labels.csv")

In [ ]:
x_image_loader._info.head()

In [ ]:
import numpy as np

global_index = 0

label_df = pd.DataFrame(columns=["filename", "category", "words", "src_file", "gt_index", "frcnn_detected_index", "FRCNN_IOU", "CRAFT_IOU", "frcnn_label"])

for image_name in tqdm.tqdm(x_image_loader._image_names):
    gt_image = x_image_loader[image_name]
    detected_image = frcnn_detector(gt_image.image)
    if detected_image is None:
        LOGGER.warning("Frcnn detection failure: %s", image_name)
        continue
    
    overlapping_info = co.check_overlapping_area(gt_image.info, detected_image.info)
    if overlapping_info is None:
        LOGGER.warning("Detected instance and GT are all disjoint: %s", image_name)
        continue

    # save gt plot
    gt_img = gt_image.draw_all_box()
    cv2.imwrite(os.path.join(output_folder_gt, image_name), cv2.cvtColor(gt_img, cv2.COLOR_RGB2BGR))
    
    # save craft plot
    det_img = detected_image.draw_all_box()
    cv2.imwrite(os.path.join(output_folder_craft_pred, image_name), cv2.cvtColor(det_img, cv2.COLOR_RGB2BGR))

    filtered_image = detected_image.keeps_index(overlapping_info["index_2nd"])

    filtered_det_img = filtered_image.draw_all_box()
    cv2.imwrite(os.path.join(output_folder_filtered_pred, image_name), cv2.cvtColor(filtered_det_img, cv2.COLOR_RGB2BGR))


    for row_idx, row in overlapping_info.iterrows():
#         if row["IOU"] < .5: # skip frcnn iou < .5
#             continue

        gt_index, detected_index = row["index_1st"], row["index_2nd"]
        detected_instance = detected_image[detected_index]
        if not detected_instance.legal:
            continue
        
        craft_detected_image = craft_detector(detected_instance.image)
        if craft_detected_image is None:
            LOGGER.warning("Craft detection failure: %s: index %d", image_name, detected_index)
            continue

        # calculate the pixels of frcnn detected bounding box
        FRCNN_GT_SIZE = np.prod(craft_detected_image.ref_image.shape)

        for craft_detected_instance in craft_detected_image:
            label_df.loc[global_index, "filename"] = f"{global_index}.jpg"
            label_df.loc[global_index, "src_file"] = image_name
            label_df.loc[global_index, "category"] = gt_image.info.loc[gt_index, "category"]
            label_df.loc[global_index, "words"] = gt_image.info.loc[gt_index, "label"]
            label_df.loc[global_index, "FRCNN_IOU"] = row["IOU"]
            label_df.loc[global_index, "gt_index"] = int(gt_index)
            label_df.loc[global_index, "frcnn_detected_index"] = int(detected_index)
            label_df.loc[global_index, "frcnn_label"] = detected_instance.info # frcnn_label

            crapped_img = craft_detected_instance.image
            # calculate the pixels of craft detected bounding box
            CRAFT_BOX_SIZE = np.prod(crapped_img.shape)
            # since the craft_box aways inside the frcnn box, the union area is equal to the frcnn box and the intersection area is equal to the craft box
            label_df.loc[global_index, "CRAFT_IOU"] = CRAFT_BOX_SIZE/FRCNN_GT_SIZE


            cv2.imwrite(os.path.join(output_folder, f"{global_index}.jpg"), cv2.cvtColor(crapped_img, cv2.COLOR_RGB2BGR))
            global_index+=1

label_df.to_csv(os.path.join(output_folder, "labels.csv"), encoding = 'big5')
print('all is done.')